In [1]:
#hide output of cell
# %%capture


!pip install pandas;
!pip install numpy;
import voila;
import ipywidgets as widgets;
from ipywidgets import interact;
from IPython.display import display;
import numpy as np;
from credsTest.SewingExport import *;
from credsTest.SummaExport import *;
from tkinter import *;
from tkinter import messagebox;
import logging


C:\Users\pgathondu\client_secret_file.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
sheets service created successfully


In [2]:

class Counter:
    def __init__(self, initial=0):
      self.value = initial
    def increment(self, amount=1):
      self.value += amount
      return self.value
    def __iter__(self, sentinal=False):
      return iter(self.increment, sentinal)
counter = Counter()  


class Pattern:
    def __init__(self, initial=False):
      self.value = initial
    def textValidate(self, last2):
      if last2[0] == last2[1]:
        display(txt3,confirm)
        if createAlert() == 'Yes Create':
          self.value = True
        else:
           self.value = False
      return self.value
    def __iter__(self, sentinal=False):
      return iter(self.textValidate, sentinal)
pat = Pattern()

class LastValue:
    def __init__(self):
      self._list = ['','']
    def record(self, txt,counter):
      self._list[counter % 2] = txt
      return self._list
    def __iter__(self, sentinal=False):
      return iter(self.record, sentinal)
class sheetInstance:
    a = {}
    def __init__(self):
      self._dict = {}
    def createSheet(self, dictSheet):
      self.a = dictSheet
      return self.a
    def __iter__(self, sentinal=False):
      return iter(self.createSheet, sentinal)
    
summaSheet = sheetInstance()
sewingSheet = sheetInstance()
last2 = LastValue()


In [3]:
txtHeader = widgets.HTML(value = '<h1>Nesting API</h1>')
txt1 = widgets.HTML(value = '<h2>Link to Spreadsheet: </h2>')
txt3 = widgets.HTML(value = """<h3> Duplicate Spreadsheets Warning!, Are you sure?\n!You've Already Created a spreadsheet!</h3>""")
txt2 = widgets.HTML(value = """<h4>Enter file Pattern: i.e if from workorder (mrp.workorder (15).csv
                    pattern -> workorder</h4>""")
txt_vbox = widgets.VBox([txt2])

In [4]:
file1 = open('img1.jpg','rb')
file2 = open('img2.jpg','rb')
pic = file1.read()
pic2 = file2.read()
img1 = widgets.Image(
    value = pic,
    format = 'JPG',
    width = '350'
)
img2 = widgets.Image(
    value = pic2,
    format = 'JPG',
    width = '300'
)

imageList = np.array([img1,img2])
anime = np.resize(imageList,100)
def f(x):
    display(anime[x])


In [5]:
get_file = widgets.RadioButtons(
    options=['readcsv/excel file', 'upload file'],
    description='File:',
    disabled=False
)

nest_type = widgets.Dropdown(
    options=['Sewing Export', 'Summa Export','Other','What file are you uploading'],
    value='What file are you uploading',
    description='Nest Type:',
    disabled=False,
)
output_drop = widgets.Output()

def dropdown_eventhandler(change):
    with output_drop:
        output_drop.clear_output()
        print('Preparing to Upload:',{nest_type.value})
        print('You will need to provide file name pattern as shown below')
        interact(f , x=5);
nest_type.observe(dropdown_eventhandler,names='value')

pattern = widgets.Text(
    value='',
    placeholder='Enter file Pattern: i.e if from workorder (mrp.workorder (15).csv -> workorder)',
    description='File Pattern:',
    disabled=False
)
          

newSheet = widgets.ToggleButtons(
    options=['Create New', 'Upload to Existing'],
    description='Create New Spreadsheet:',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Default', 'Provide sheet id'],
#     icons=['check'] * 3
)
filters = widgets.ToggleButtons(
    options=['Default Filters', 'User Defined'],
    description='Pivot Filters:',
    disabled=True,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Use default', 'Provide columns to filter'],
#     icons=['check'] * 3
)

confirm = widgets.ToggleButtons(
    options=['Yes Create', 'No'],
    description='Alert:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Use default', 'Provide columns to filter'],
#     icons=['check'] * 3
)

In [6]:
def createAlert():
#     root = Tk()
#     root.lift()
#     root.attributes('-topmost',True)
# #     root.after_idle(root.attributes,'-topmost',False)
    
#     root.withdraw()
#     root.geometry("300x200")
#     # root.after(3000,root.destroy)
#     a =messagebox.askquestion("Multiple Spreadsheets Warning!", "Are you sure?\n!You've Already Created a spreadsheet!")

#     if a:
#         root.destroy()
    return confirm.value
def check():
    return pattern.value != '' and pat.textValidate(last2.record(pattern.value,counter.value))

def loop():
    if createAlert() == 'yes':
        loop()
    else:
        print('all done')
def sheetType():
    if nest_type.value == 'Sewing Export':
        try: 
            print('creating pivot for Sewing covers')
            sewingSheet.createSheet(createSewingPivot(pattern.value));
            display(txt1,sewingSheet.a['Url'])
            
            summaryTable(sewingSheet.a)
        except:
            print('Invalid file name pattern entered')

    elif nest_type.value == 'Other':
        print('This is not supported Currently')
    elif nest_type.value == 'Summa Export':
        try: 
            print('creating pivot for Summa')
            summaSheet.createSheet(createSummaPivot(pattern.value));
            display(txt1,summaSheet.a['Url'])
            summaryTable(summaSheet.a)
        except:
            logging.exception("message")
            print('Invalid file name pattern entered')

def summaryTable(nest):
    time_stamp = file_time_stamp(pattern.value)['time']
    file_name_read = file_time_stamp(pattern.value)['file']
    df = pd.DataFrame({'file name': [str(file_name_read)],'file_write_date':[time_stamp]})
    add_sheets(nest['gsheetId'], 'Summary Table')
    writeDataToSheetDf('Summary Table',nest['gsheetId'],df)
        

In [7]:
from functools import partial
createNest = widgets.Button(
    description = 'Create',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)
output = widgets.Output()
testing = 'hi'

def on_button_clicked(counter,event):
    with output:
        output.clear_output()
        counter.increment()
        can_create = check()
        print('Click Counter:',counter.value)
        if can_create:
            sheetType()
        else:
            print('Creation Canceled due to empty file name or duplicate')

 
createNest.on_click(partial(on_button_clicked,counter))
vbox_result = widgets.VBox([createNest,output])

file_type = widgets.VBox([nest_type,output_drop])


vbox_rest = widgets.VBox([txtHeader,file_type,get_file,txt2,pattern,newSheet,filters,vbox_result])


In [8]:
page = widgets.HBox([vbox_rest])
display(page)

